In [1]:
import numpy as np

from limlam_mocker import limlam_mocker as llm
import lnn as lnn

%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
### set the location of the halo catalogs and maps
haloLoc = '../catalogues2/'
mapLoc = '../maps2/test/'

In [4]:
### what noise level to use
noise = 0

### to use base coefficients for the model or not
default = True

### make sure the map directory exists
lnn.checkDirectoryPath(mapLoc)

### get the subfield names
subFields = lnn.loadSubFields(haloLoc)

### number of maps to make
numb_maps = 5796


In [5]:
### start making random maps
for i in range(numb_maps)[:2]:
    lnn.make_random_map(subFields, haloLoc, mapLoc, default, noise=noise)

    ### print every 100 maps where we are in the process
    if i%100 == 0:
            print('Finished making map {} out of {} ({:.3f}%)'.format(i, numb_maps, i/numb_maps*100))

Finished making map 0 out of 5796 (0.000%)


In [33]:
mapLoc = '../maps2/random_Li/'
mapLoc_noise = '../maps2/random_noisy_Li/'
mapLoc_different = '../maps2/random_maps2/'
catLoc = '../catalogues2/'
modelLoc = '../models2/'
modelLoc2 = '../models3/'

subFields = lnn.loadBaseFNames(mapLoc_different)
base = [mapLoc_different + s for s in subFields]

In [45]:
fName = base[0] + '_lum.npz'
data = lnn.loadData(fName)

In [44]:
model = model_from_file_name(subFields[0])

### save the file
np.savez(str(fName),
    logBinCent  =   data['logBinCent'],
    numberCt    =   data['numberCt'],
    lumFunc     =   data['lumFunc'],
    model       =   model)

In [59]:
from datetime import datetime

In [36]:
def model_from_file_name(file_name):
    if 'Li' in file_name:
        model = 'Li'
    elif 'Padmanabhan' in file_name:
        model = 'Padmanabhan'
    elif 'Breysse' in file_name:
        model = 'Breysse'
    else:
        model = 'None'
        
    return(model)

In [51]:
subFields = lnn.loadBaseFNames(mapLoc_different)

In [66]:
subFields[0]

'COMAP_z2.39-3.44_1140Mpc_seed_13827_rotate_0_subfield_21__Breysse_2.000e-06_1.000e+00'

In [68]:
map_loc = '../maps2/random_noisy_Li/'
subFields = lnn.loadBaseFNames(map_loc)
startTime = datetime.now()

for sub in subFields:
    file_name = map_loc + sub + '_lum.npz'
    model = model_from_file_name(file_name)

    data = lnn.loadData(file_name)

    np.savez(str(file_name),
        logBinCent  =   data['logBinCent'],
        numberCt    =   data['numberCt'],
        lumFunc     =   data['lumFunc'],
        model       =   model)
    
print((datetime.now() - startTime).total_seconds()/20.0)

15.8630201


In [46]:
list(data)

['lumFunc', 'model', 'logBinCent', 'numberCt']

In [47]:
dict(data)

{'logBinCent': array([  3.43332002e+03,   4.04708995e+03,   4.77058270e+03,
          5.62341325e+03,   6.62870316e+03,   7.81370738e+03,
          9.21055318e+03,   1.08571112e+04,   1.27980221e+04,
          1.50859071e+04,   1.77827941e+04,   2.09617999e+04,
          2.47091123e+04,   2.91263265e+04,   3.43332002e+04,
          4.04708995e+04,   4.77058270e+04,   5.62341325e+04,
          6.62870316e+04,   7.81370738e+04,   9.21055318e+04,
          1.08571112e+05,   1.27980221e+05,   1.50859071e+05,
          1.77827941e+05,   2.09617999e+05,   2.47091123e+05,
          2.91263265e+05,   3.43332002e+05,   4.04708995e+05,
          4.77058270e+05,   5.62341325e+05,   6.62870316e+05,
          7.81370738e+05,   9.21055318e+05,   1.08571112e+06,
          1.27980221e+06,   1.50859071e+06,   1.77827941e+06,
          2.09617999e+06,   2.47091123e+06,   2.91263265e+06,
          3.43332002e+06,   4.04708995e+06,   4.77058270e+06,
          5.62341325e+06,   6.62870316e+06,   7.81370738